In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
#import urllib
from PIL import Image
import numpy as np

In [24]:
def CheckExist(xpath, wait):   
    locator=(By.XPATH,xpath)
    try: 
        WebDriverWait(driver,wait).until(EC.presence_of_element_located(locator))
        return('ok')
    except TimeoutException:
        return('timeout')
    return 

In [43]:
def LoginSubmit():
    userInput = driver.find_element(By.XPATH,'/html/body/snapon-app/snapon-base/div/div/snapon-login/div[2]/div/form/div/div[2]/div[6]/input')
    userInput.send_keys('TIS60user1')
    pwdInput = driver.find_element(By.XPATH,'/html/body/snapon-app/snapon-base/div/div/snapon-login/div[2]/div/form/div/div[2]/div[7]/input')
    pwdInput.send_keys('Z8aj9Liz')
    loginbut = driver.find_element(By.XPATH,'/html/body/snapon-app/snapon-base/div/div/snapon-login/div[2]/div/form/div/div[2]/div[10]/button/span')
    loginbut.click()

    PopMessage = '/html/body/snapon-app/snapon-epc/div/split-panel/split-area[1]/epc-navigation/notification-dialog/p-dialog/div/div/div[1]/div/button/timesicon'
    check = CheckExist(PopMessage,15)
    if check == 'ok':
        PopMessage = driver.find_element(By.XPATH,PopMessage)
        PopMessage.click()
        return ('ok')
    else:
        return('timeout')

In [42]:
def VINSearch(VINCode):
    inputVIN = driver.find_element(By.XPATH,'/html/body/snapon-app/snapon-epc/div/div[1]/div[2]/epc-toolbar/p-toolbar/div/div[1]/p-dropdown/div/input')
    inputVIN.send_keys(VINCode)
    clickSearch = driver.find_element(By.XPATH,'/html/body/snapon-app/snapon-epc/div/div[1]/div[2]/epc-toolbar/p-toolbar/div/div[1]/button')
    clickSearch.click()


In [75]:
def getMainCat():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    TableFrame = soup.find_all('div',{'class':'ag-center-cols-container', 'ref':'eContainer','role':'rowgroup'})[0]
    MainCat = TableFrame.findAll('div',{'class':'ag-cell-wrapper'})
    MainCat = '\n'.join([x.text for x in list(MainCat)])
    return(MainCat)

In [111]:
def ClickOnCat(clickText):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    target_text = clickText
    element = soup.findAll('span', text=target_text)

    if element:
        # Find the parent div with the necessary attributes to locate it in Selenium
        parent_div = element[0].find_parent('div', class_='ag-row')

        if parent_div:
            # Get the unique attribute to locate this element using Selenium
            row_id = parent_div.get('row-id')
            # Create the XPath using the row-id
            xpath = f"//div[@row-id='{row_id}']"

            # Use Selenium to find the element and click it
            selenium_element = driver.find_element(By.XPATH, xpath)
            actions = ActionChains(driver)
            actions.move_to_element(selenium_element).click().perform()

In [144]:
def findSubCat():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    FrameSubCat = soup.findAll('div',{'class':'ag-center-cols-container', 'role':'rowgroup'})[1]
    FrameSubCat = list(FrameSubCat.findAll('div',{'class':'ag-cell-wrapper'}))
    FrameSubCat = '\n'.join([x.text for x in FrameSubCat])
    return(FrameSubCat)

In [281]:
def getTableResult():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    CrossCatKey = list(soup.findAll('div',{'class':'ag-cell-value ag-cell ag-cell-not-inline-editing ag-cell-normal-height fixed-height','col-id':'crossCatKey'}))
    FormattedPartNo = list(soup.findAll('div',{'class':'ag-cell-value ag-cell ag-cell-not-inline-editing ag-cell-normal-height fixed-height','col-id':'formattedPartNumber'}))
    Position = list(soup.findAll('div',{'class':'ag-cell-value ag-cell ag-cell-not-inline-editing ag-cell-normal-height fixed-height','col-id':'position','aria-colindex':'7'}))
    Description = list(soup.findAll('div',{'class':'ag-cell-value ag-cell ag-cell-not-inline-editing ag-cell-normal-height','col-id':'description'}))

    CrossCatKeyX = []
    for idx, x in enumerate(CrossCatKey):
        gotX = x.findAll('suppress-value-renderer',{'class':'ng-star-inserted'})
        if len(gotX)>0:
            CrossCatKeyX.append(gotX[0].text)
    FormattedPartNo = [x.text for x in FormattedPartNo]
    Position = [x.text for x in Position]
    Description = [x.text for x in Description]
    dataTable = { 'FIG_KEY':CrossCatKeyX,
                'PART_NO' :FormattedPartNo,
                'POSITION':Position,
                'DESCRIPTION':Description }
    dataTable = pd.DataFrame(dataTable)
    return(dataTable)


In [97]:
VIN = 'MP1TFR86JEG007983'
driver = webdriver.Chrome()
driver.get('https://isuzu.snaponepc.com/epc/#/')
xpath = "/html/body/snapon-app/snapon-base/div/div/snapon-login/div[2]/div/form/div/div[2]/div[6]/input"
waitforLogin = CheckExist(xpath,10)
if waitforLogin == 'ok':
    print('login '+LoginSubmit())
    VINSearch(VIN)
    time.sleep(3)
    print('Main Category : \n'+ getMainCat())
else:
    print('timeout')


login ok
Main Category : 
0 - ENGINE,EMISSION,ENGINE ELECTRICAL
1 - FUEL TANK,COOLING,AIR INTAKE,EXHAUST SYSTEM
2 - CLUTCH,TRANSMISSION,TRANS AXLE
3 - BRAKE,BRAKES SYSTEM
4 - PROPELLER SHAFT,AXLES,STEERING,SUSPENSION
5 - FRAME,CAB MOUNTING
6 - EXTERIOR BODY,GLASS MOUNTING,DOOR TRIM,MOLDING
7 - INTERIOR TRIM,EMBLEMS,LABELS,SEATS,MIRRORS
8 - CHASSIS ELECTRICAL,HEATING,AIR CONDITIONING,WIPERS,RADIO
9 - TOOLS


In [115]:
TobeClick = '6 - EXTERIOR BODY,GLASS MOUNTING,DOOR TRIM,MOLDING'
ClickOnCat(TobeClick)

/var/folders/z3/9bpskt7j7tvfmp076hkrvm1h0000gn/T/ipykernel_81922/27207204.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  element = soup.findAll('span', text=target_text)


In [145]:
print(findSubCat())

6-01 - FRONT BUMPER
6-03 - RADIATOR GRILLE
6-05 - FRONT PANEL
6-07 - FRONT WINDOW GLASS AND WEATHERSTRIP
6-10 - ENGINE HOOD AND LOCK
6-14 - DASH PANEL AND FENDER SKIRT
6-20 - SIDE PANEL AND STEP
6-30 - FRONT DOOR AND HINGE
6-31 - FRONT DOOR GLASS AND REGULATOR
6-32 - FRONT DOOR LOCK AND HANDLE
6-35 - FRONT DOOR TRIM
6-43 - SIDE TRIM
6-45 - MOLDING AND STRIPE
6-47 - MUD GUARD AND INSULATOR
6-50 - REAR DOOR AND HINGE
6-51 - REAR DOOR GLASS AND REGULATOR
6-52 - REAR DOOR LOCK AND HANDLE
6-55 - REAR DOOR TRIM
6-60 - ROOF PANEL
6-66 - HEAD LINING AND TRIM
6-70 - FLOOR PANEL
6-75 - MAT AND CARPET
6-76 - FLOOR EQUIPMENT AND SEALING
6-78 - FLOOR HOLE PLUG
6-80 - BACK PANEL
6-82 - BACK GLASS AND WEATHERSTRIP
6-84 - BACK TRIM
6-86 - TRUNK OPENER AND FUEL LID
6-90 - REAR BUMPER


In [279]:
TobeClick = '6-90 - REAR BUMPER'
ClickOnCat(TobeClick)

/var/folders/z3/9bpskt7j7tvfmp076hkrvm1h0000gn/T/ipykernel_81922/27207204.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  element = soup.findAll('span', text=target_text)


In [283]:
getTableResult()

,FIG_KEY,PART_NO,POSITION,DESCRIPTION
0,690-001(A),8-98153-274-4,,BUMPER; RR
1,690-002,8-98062-142-0,,BOLT; BACK BAR
2,690-225,8-98102-359-2,,"PAD; STEP,RR BUMPER"
3,690-349,8-98238-273-0,R,SHELL; RR BUMPER
4,,8-98238-274-0,L,
